In [1]:
import pandas as pd
import sparql_dataframe

wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"


#### The "Characters" queries
##### Bechdel test query: *how many of the [selected and tested for Bechdel] movies have **male** directors?*
To answer this query (and the following one, regarding character dialogues) we gather data from the `dialogue_bechdel.csv` file.

We first **read the CSV file as a dataframe and clean it**, dropping all the movies which have not been tested for the Bechdel test: these movies will have a `NaN` value under the `bechdel_rating` column.

We then create an **empty list `film_list_bechdel`**, containing tuples representing the IMDB id of the movie (`imdbid` column) and its result in the Bechdel test (column `bechdel_rating`).
If the `bechdel_rating` is...:
- 0 &rarr; FAILED the first criteria
- 1 &rarr; FAILED the second criteria
- 2 &rarr; FAILED the third criteria
- 3 &rarr; PASSED the test (passed all three criteria) 

As our starting IMDB's ids are actually different than those present in wikipedia (which have a suffix differentiating between titles, names, companies, events, news...), before populating the `film_list_bechdel` we need to process the ids and add the suffix.

We do so with the appropriate function `createIMDBid`.

Then, we populate the `film_list_bechdel` and measure its length: this is the total number of movies which have been tested for the Bechdel test (72).

In [2]:
df = pd.read_csv('../data/dialogue/dialogue_bechdel.csv')

def createIMDBid(code):
    if len(str(code)) == 5:
        return "tt00"+str(code)
    elif len(str(code)) == 6:
        return "tt0"+str(code)
    elif len(str(code)) == 7:
        return "tt"+str(code)

bechdel_df = df.dropna(axis=0, subset=["bechdel_rating"])

film_list_bechdel = list()

for idx, row in bechdel_df.iterrows():
    imdb_id = createIMDBid(row["imdbid"])
    tuple = (imdb_id, row["bechdel_rating"])
    film_list_bechdel.append(tuple)

n_films_B = len(film_list_bechdel)    # 72
print("Total number of movies tested for Bechdel test:\t",n_films_B)

Total number of movies tested for Bechdel test:	 72



We create a `ids_tpl_bechdel` tuple (to be used in the SPARQL query) containing only the formatted IMDB's ids taken from the `film_list_bechdel`.

Finally, we query the SPARQL endpoint, selecting only the movies from our list which have a **male director** (specified by the Wikidata class `wd:Q6581097`).

We use the `FILTER` and `IN` clauses to run the query on all the IMDB's ids contained in our list without having to open the query connection multiple times (as experienced, that will overwork Wikidata's query service and the IP of the computer used to run the query will be momentarily banned).

In [3]:
ids_tpl_bechdel = ()

for tpl in film_list_bechdel:
    ids_tpl_bechdel = ids_tpl_bechdel + (tpl[0],)

# SPARQL
query_gender_director = '''
        SELECT ?imdb ?Movie ?Director
        WHERE {{
            ?movie wdt:P345 ?imdb ;
                    wdt:P57 ?director ;
                    rdfs:label ?Movie .
            ?director rdfs:label ?Director ;
                        wdt:P21 wd:Q6581097 .
            FILTER ((lang(?Director) = "en") && (lang(?Movie) = "en")) .
            FILTER (?imdb IN {list}) .
        }}
    '''

result_bechdel_query = sparql_dataframe.get(
    wikidata_endpoint, query_gender_director.format(list=ids_tpl_bechdel), True)

Now, as we want to add the outcome of the Bechdel test of the result of the query, we create another dataframe `add_bech_df` from the list of tuples `film_list_bechdel` (in which the first element of each tuple is the formatted IMDB's id, and the second element is the outcome of the Bechdel test).

We then merge the two dataframes `result_bechdel_query` and `add_bech_df` together, using the `imdb` column as merging point.

Now, if we count the number of rows of the updated `result_bechdel_query`, we will have the **total number of male directors of the movies tested for the Bechdel test**.
Please notice how this number is actually higher than the total number of movies tested for the Bechdel test (`n_films_B`): this is because some movies will have more than one director.

The result of this query means that **no matter the result of the Bechdel test, all the movies which have been tested for it have male directors**.

In [4]:
add_bech_df = pd.DataFrame(film_list_bechdel, columns=[
                           "imdb", "Bechdel_result"])

result_bechdel_query = result_bechdel_query.merge(
    add_bech_df, left_on="imdb", right_on="imdb")


total_Mdirectors = (len(result_bechdel_query.index))  # 79
print("Total number of movies tested for Bechdel test WITH male director(s):\t",total_Mdirectors)



Total number of movies tested for Bechdel test WITH male director(s):	 79


##### Characters dialogue query: *how many of the [selected] films have **male** directors?*

Even for this query we are using the data from the `dialogue_bechdel.csv` file.

The reasoning behind this query is more or less the same as the previous one:
1. **Read the CSV file as a dataframe and clean it** from all the movies that have no dialogue analysis (using the `.dropna` instruction, as they will have a `NaN` value under the `male_percen` column)
2. Create a **`film_list_dlg` of tuples containing the IMDB id of the movie** (`imdbid` column) and the **information on the dialogues** (`male_percen` and `nonmale_percentage` columns); then, measure its length: this is the total number of movies which have dialogue analysis
3. Use the previously defined `createIMDBid` function to add the Wikidata's suffix to our starting IMDB's ids


In [5]:

film_list_dlg = list()

dlg_df = df.dropna(axis=0, subset=["male_percen"])

for idx, row in dlg_df.iterrows():
    imdb_id = createIMDBid(row["imdbid"])
    tuple = (imdb_id, row["male_percen"], row["nonmale_percentage"])
    film_list_dlg.append(tuple)

n_films = len(film_list_dlg)    # 66
print("Total number of movies with a dialogue analysis:\t",n_films)

Total number of movies with a dialogue analysis:	 66


4. Create the `ids_tpl_dlg` tuple with only the formatted IMDB's ids (taken from the `film_list_dlg`)
5. Query the SPARQL endpoint selecting **the writers for each movie (regardless of their gender) and their gender** "value"
    - The use of the `OPTIONAL` clause was necessary as it seems not all writers have the gender information available

In [6]:
ids_tpl_dlg = ()

for tpl in film_list_dlg:
    ids_tpl_dlg = ids_tpl_dlg + (tpl[0],)

# SPARQL
query_gender_director = '''
    SELECT ?imdb ?Movie ?Writer ?Gender
    WHERE {{
        ?movie wdt:P345 ?imdb ;
                wdt:P58 ?writer ;
                rdfs:label ?Movie .
        ?writer rdfs:label ?Writer .
        OPTIONAL {{
            ?writer wdt:P21 ?gender .
            ?gender rdfs:label ?Gender .
            FILTER ( (lang(?Gender) = "en") )
        }}
        FILTER ( (lang(?Writer) = "en") && (lang(?Movie) = "en"))
        FILTER ( ?imdb IN {list} )
}}
'''

result_dlg_query = sparql_dataframe.get(
    wikidata_endpoint, query_gender_director.format(list=ids_tpl_dlg), True)
result_dlg_query

,imdb,Movie,Writer,Gender
0,tt1289401,Ghostbusters,Katie Dippold,female
1,tt0081573,Superman II,Leslie Newman,female
2,tt0078346,Superman,Leslie Newman,female
3,tt1877832,X-Men: Days of Future Past,Jane Goldman,female
4,tt0241527,Harry Potter and the Philosopher's Stone,J. K. Rowling,female
...,...,...,...,...
149,tt0034583,Casablanca,Howard Koch,male
150,tt0055614,West Side Story,Ernest Lehman,male
151,tt0059742,The Sound of Music,Ernest Lehman,male
152,tt0067093,Fiddler on the Roof,Joseph Stein,male


6. Add the outcome of the dialogue analysis through a new dataframe `add_dlg_df`, created from the list of tuples `film_list_dlg`
7. Merge the two dataframes `result_dlg_query` and `add_dlg_df`
8. Save the dataframe in a CSV file

In [7]:
add_dlg_df = pd.DataFrame(film_list_dlg, columns=[
                           "imdb", "male_percentage", "nonmale_percentage"])

# Merge the two dataframes together using the IMDB ids columns
result_dlg_query = result_dlg_query.merge(
    add_dlg_df, left_on="imdb", right_on="imdb")

result_dlg_query
# result_dlg_query.to_csv('data/sparql/dlg.csv')

,imdb,Movie,Writer,Gender,male_percentage,nonmale_percentage
0,tt1289401,Ghostbusters,Katie Dippold,female,86.68,13.32
1,tt1289401,Ghostbusters,Paul Feig,male,86.68,13.32
2,tt0081573,Superman II,Leslie Newman,female,75.89,24.11
3,tt0081573,Superman II,David Newman,male,75.89,24.11
4,tt0081573,Superman II,Mario Puzo,male,75.89,24.11
...,...,...,...,...,...,...
149,tt0103776,Batman Returns,Daniel Waters,male,61.76,38.24
150,tt0103776,Batman Returns,Wesley Strick,male,61.76,38.24
151,tt0055614,West Side Story,Ernest Lehman,male,67.31,32.69
152,tt0059742,The Sound of Music,Ernest Lehman,male,25.29,74.71


Now we can quickly compare the number of male and female writers in our selection of the movies. We do so by simply iterating through the `result_dlg_query` dataframe and update the number of writers (either `n_Mwriters` or `n_Fwriters`) depending on the value under the column `Gender`. We also print out the total number of writers in our 66 selected movies.

The difference is clear and pretty straightforward.

In [8]:
n_writers = (len(result_dlg_query.index))
print("Total number of writers of the selected 66 movies:\t",n_writers) # 154


n_Mwriters = 0
n_Fwriters = 0
for idx, row in result_dlg_query.iterrows():
    if row["Gender"] == 'male':
        n_Mwriters += 1
    else:
        n_Fwriters += 1

print("Number of male writers\t:", n_Mwriters)  # 143
print("Number of female writers\t:", n_Fwriters)    # 11


""" 143 : 154 = x : 100 
x = (143*100)/154 = 92.85

11 : 154 = y : 100
y = (11*100)/154 = 7.14 """

# result_dlg_query.to_csv('data/sparql/dlg.csv')


Total number of writers of the selected 66 movies:	 154
Number of male writers	: 143
Number of female writers	: 11


' 143 : 154 = x : 100 \nx = (143*100)/154 = 92.85\n\n11 : 154 = y : 100\ny = (11*100)/154 = 7.14 '

#### Gaze score queries

For these queries, the data used comes from the `final_scores_df.csv` CSV file.
Again, even in this case the reasoning is always the same as before.


##### GS query 1: *To what genre belong the top 10 films in the gaze score ranking?*

1. **Read the CSV file as a dataframe and clean it** from all the movies that have no male gaze score (using the `.dropna` instruction, as they will have a `NaN` value under the `gaze_score` column); then, **sort it** depending on the male gaze value (`MG_df`) and then **select only the top 10 movies** (`topMG_df`)
2. Create a **`film_list_mg1` of tuples containing the IMDB id of the movie** (`imdbid` column) and the **male gaze score** (`gaze_score` column); then, measure its length: this is the total number of movies which have a male gaze score
3. Use the previously defined `createIMDBid` function to add the Wikidata's suffix to our starting IMDB's ids
4. Create the `ids_tpl_mg1` tuple with only the formatted IMDB's ids (taken from the `film_list_mg1`)
5. Query the SPARQL endpoint selecting **the genres for each movie** of the movies in the list

In [12]:
df_mg = pd.read_csv('../data/final_scores/final_scores_df.csv')

MG_df = df_mg.dropna(axis=0, subset=["gaze_score"])

MG_df.sort_values(by="gaze_score", ascending=False, inplace=True, ignore_index=True)

topMG_df = MG_df.head(10)


film_list_mg1 = list()

for idx, row in topMG_df.iterrows():
    imdb_id = createIMDBid(row["imdbid"])
    tuple = (imdb_id, row["gaze_score"])
    film_list_mg1.append(tuple)

print("Top 10 movies of the male gaze score ranking:\t",len(film_list_mg1))

ids_tpl_mg1 = ()

for tpl in film_list_mg1:
    ids_tpl_mg1 = ids_tpl_mg1 + (tpl[0],)

# SPARQL
query_10_mg = '''
    SELECT ?imdb ?Movie ?Genre
    WHERE {{
        ?movie wdt:P345 ?imdb ;
                wdt:P136 ?genre ;
                rdfs:label ?Movie .
        ?genre rdfs:label ?Genre .
        FILTER ( (lang(?Movie) = "en") && (lang(?Genre) = "en"))
        FILTER ( ?imdb IN {list} )
    }}
'''

result_mg1_query = sparql_dataframe.get(wikidata_endpoint, query_10_mg.format(list=ids_tpl_mg1),True)
result_mg1_query

Top 10 movies of the male gaze score ranking:	 10


,imdb,Movie,Genre
0,tt0145487,Spider-Man,drama film
1,tt0120755,Mission: Impossible 2,action film
2,tt0145487,Spider-Man,science fiction film
3,tt0120755,Mission: Impossible 2,spy film
4,tt0145487,Spider-Man,superhero film
5,tt0145487,Spider-Man,coming-of-age fiction
6,tt0120755,Mission: Impossible 2,thriller film
7,tt1877832,X-Men: Days of Future Past,action film
8,tt1877832,X-Men: Days of Future Past,science fiction film
9,tt1877832,X-Men: Days of Future Past,superhero film


6. Add the male gaze score through a new dataframe `add_mg1_df`, created from the list of tuples `film_list_mg1`
7. Merge the two dataframes `result_mg1_query` and `add_mg1_df`
8. Save the dataframe in a CSV file

In [13]:
add_mg1_df = pd.DataFrame(film_list_mg1,columns=["imdb", "gaze_score"])

result_mg1_query = result_mg1_query.merge(add_mg1_df,left_on="imdb",right_on="imdb")
# result_mg1_query.to_csv('../data/sparql/mg1.csv')
result_mg1_query

,imdb,Movie,Genre,gaze_score
0,tt0145487,Spider-Man,drama film,60.795026
1,tt0145487,Spider-Man,science fiction film,60.795026
2,tt0145487,Spider-Man,superhero film,60.795026
3,tt0145487,Spider-Man,coming-of-age fiction,60.795026
4,tt0120755,Mission: Impossible 2,action film,71.293404
5,tt0120755,Mission: Impossible 2,spy film,71.293404
6,tt0120755,Mission: Impossible 2,thriller film,71.293404
7,tt1877832,X-Men: Days of Future Past,action film,58.120616
8,tt1877832,X-Men: Days of Future Past,science fiction film,58.120616
9,tt1877832,X-Men: Days of Future Past,superhero film,58.120616


##### GS query 2: *Is there any correlation between rank in the gaze score ranking, box-office and production costs?*

1. Use the `MG_df` already cleaned and sorted from before
2. Create a **`film_list_mg2` of tuples containing the IMDB id of the movie** (`imdbid` column) and the **male gaze score** (`gaze_score` column); then, measure its length: this is the total number of movies which have a male gaze score
3. Use the previously defined `createIMDBid` function to add the Wikidata's suffix to our starting IMDB's ids
4. Create the `ids_tpl_mg2` tuple with only the formatted IMDB's ids (taken from the `film_list_mg2`)
5. Query the SPARQL endpoint selecting **the production costs and the box office** of the movies in the list
    - Again, the `OPTIONAL` clauses were necessary for the lack of information regarding production costs and box office for some of the movies in the list

In [14]:
film_list_mg2 = list()

for idx, row in MG_df.iterrows():
    imdb_id = createIMDBid(row["imdbid"])
    tuple = (imdb_id, row["gaze_score"])
    film_list_mg2.append(tuple)

print("Total number of movies with a male gaze score:\t",len(film_list_mg2))

ids_tpl_mg2 = ()

for tpl in film_list_mg2:
    ids_tpl_mg2 = ids_tpl_mg2 + (tpl[0],)


# SPARQL
query_costs_mg = '''
SELECT ?imdb ?Movie ?ProductionCosts ?BoxOffice 
WHERE {{
  ?movie wdt:P345 ?imdb ;
        rdfs:label ?Movie .
  OPTIONAL {{
    ?movie wdt:P2130 ?ProductionCosts .
  }}
  OPTIONAL {{
    ?movie wdt:P2142 ?BoxOffice .
    ?statement ps:P2142 ?BoxOffice .
    ?statement pq:P3005 ?validity .
    }}
  FILTER ( (lang(?Movie) = "en") && ((?validity = wd:Q30) || (?validity = wd:Q49)) )
  FILTER NOT EXISTS {{ ?statement pq:P1264 ?o }}
  FILTER ( ?imdb in {list} )
}}
'''

result_mg2_query = sparql_dataframe.get(wikidata_endpoint, query_costs_mg.format(list=ids_tpl_mg2),True)
result_mg2_query

Total number of movies with a male gaze score:	 80


EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT ?imdb ?Movie ?ProductionCosts ?BoxOffice \nWHERE {\n  ?movie wdt:P345 ?imdb ;\n        rdfs:label ?Movie .\n  OPTIONAL {\n    ?movie wdt:P2130 ?ProductionCosts .\n  }\n  OPTIONAL {\n    ?movie wdt:P2142 ?BoxOffice .\n    ?statement ps:P2142 ?BoxOffice .\n    ?statement pq:P3005 ?validity .\n    }\n  FILTER ( (lang(?Movie) = "en") && ((?validity = wd:Q30) || (?validity = wd:Q49)) )\n  FILTER NOT EXISTS { ?statement pq:P1264 ?o }\n  FILTER ( ?imdb in (\'tt0120755\', \'tt0080684\', \'tt0062622\', \'tt0056172\', \'tt0103776\', \'tt0145487\', \'tt0109830\', \'tt1877832\', \'tt0071230\', \'tt0096895\', \'tt0078346\', \'tt0316654\', \'tt0092644\', \'tt0848228\', \'tt0121766\', \'tt0120338\', \'tt0086190\', \'tt0112462\', \'tt0075148\', \'tt0119654\', \'tt0096438\', \'tt0076759\', \'tt0064115\', \'tt0034583\', \'tt0061722\', \'tt0068646\', \'tt0103064\', \

6. Add the male gaze score through a new dataframe `add_mg2_df`, created from the list of tuples `film_list_mg2`
7. Merge the two dataframes `result_mg2_query` and `add_mg2_df`
8. Save the dataframe in a CSV file

In [ ]:
add_mg2_df = pd.DataFrame(film_list_mg2,columns=["imdb", "gaze_score"])

result_mg2_query = result_mg2_query.merge(add_mg2_df,left_on="imdb",right_on="imdb")
result_mg2_query

# result_mg2_query.to_csv('../data/sparql/mg2.csv')